In [1]:
import subprocess
import warnings
import numpy as np
import pandas as pd
import time
import math
from scipy.stats import bernoulli as bn
from cyvcf2 import VCF, Writer, Variant
from typing import *
import os

In [2]:
def random_delete(rate: float = 0.01, activate: bool = False) -> bool:  #where is arr??
    """
    Delete randomly genotypes in a variant call.
    :param arr: variant.genotypes instance
    :param rate: proportion of genoyptes to delete over the call
    :param activate: activation of the deletion function
    :return: array of boolean indicating if the genotypes have to be deleted (1) or not (0).
    """
    if activate:
        flag: bool = bn.rvs(p=rate, size=1)    #Generate A Bernoulli discrete random variable : 0 or 1
    else:
        flag: bool = False

    return flag     # [1] or [0]

In [3]:
def split_pools(idv_id: List[str], pool_size: int, seed: object = None) -> Tuple[List[str], List[str]]:
    """
    Assign samples to pools and output the remaning samples that could not be assigned (to few samples).
    :param idv_id: sample's identiifer in the VCF-file.
    :param pool_size: size of group expected (e.g. 16 if 4*4 pooling design)
    :return: samples identifiers grouped and remaining samples
    """
    '''
    Caculate the number of pools based on sample(idv) number and pool size, randomly assign samples into these pools,
    return pools in list form and return the not-assigned samples.
    '''

    idv_nb = len(idv_id)
    nb_pool, left = divmod(idv_nb, pool_size)
    print('Number of {}-sized pools to create, number of single samples remaining: '.format(pool_size),
          nb_pool, left)
    # Create random pools
    if seed is not None:
        np.random.seed(seed)
    pools = np.random.choice(idv_id, size=(nb_pool, pool_size), replace=False)
    left = np.isin(idv_id, pools.flatten())
    singles = np.extract(~left, idv_id)
    pools = pools.tolist()   #Return a copy of the array data as a (nested) Python list.

    return pools, singles

In [4]:
class SNPsPool(np.ndarray):
    """
    Simulates the different steps of a genotype pooling process.
    Builds the pooling design matrix.
    Encodes pooled genotypes per group.
    Decodes encoded genotypes of the pools back to individual genotypes.
    """
    def __new__(cls,
                shape: tuple = (4, 4),
                id_len: int = 8,
                pools_nb: int = 8,
                pools_size: int = 4) -> np.ndarray:
        """
        Define the basic structure for a pool i.e.
        a squared matrix to fill with the variables IDs/GT/GL.
        :param shape: tuple, shape of the pool
        :param id_len: max number of char of the variables IDs
        :return: a matrix with dims 'shape', filled with str types
        """
        cls.id_len = id_len
        id = 'U' + str(cls.id_len)
        cls.pools_nb = pools_nb
        cls.pools_size = pools_size
        return np.empty_like(super(SNPsPool, cls).__new__(cls, shape),
                             dtype=id)

    def design_matrix(self, random: bool = False) -> np.ndarray:
        """
        That function is not intended to be called explicitly.
        :param random: bool for dispatching idv randomly in the matrix?
        :return: design matrix. Numpy array.
        """
        pools_size: int = self.pools_size
        design: np.ndarray = np.zeros((self.pools_nb, self.size), dtype=int)
        if not random:
            for i in range(int(self.pools_nb/self.ndim)):
                j = i * pools_size
                design[i, j:j+pools_size] = [1]*pools_size
            for i in range(int(self.pools_nb/self.ndim), self.pools_nb):
                j = i - pools_size
                design[i,
                       [j+k*pools_size for k in range(pools_size)]] = 1
        return design

    def set_subset(self, subset: np.ndarray) -> np.ndarray:
        """
        Fills the pooling matrix according to the (default) design
        and the input list of samples.
        :param subset: 1D-nparray-like object with the variables IDs
        :return: pooling matrix with samples' names.
        """
        self.__setattr__('subset', subset)  # from split_pools
        sub = self.__getattribute__('subset')
        try:
            for i in range(self.shape[0]):
                self[i, :] = sub[:self.shape[1]]
                sub = sub[self.shape[1]:]
        except Exception as exc:
            if len(self.subset) > self.size:
                raise ValueError('The input you gave is too long') from exc
            if len(self.subset) < self.size:
                raise ValueError('The input you gave is too short') from exc
            if type(self.subset) != np.ndarray and type(self.subset) != list:
                raise TypeError('The input is not a 1D-array-like') from exc
            if len(self.subset) > 0 and type(self.subset[0]) != str:
                raise TypeError('The input does not contain str-type elements') from exc

        return self

    def get_subset(self) -> np.ndarray:
        """
        Flatten the matrix of pooled samples identifiers.
        :return: flattened array of samples identifiers or genotypes.
        """
        ids = self.flatten()  # .reshape(1, self.size)
        return ids

    def pools_list(self) -> List[str]:
        """
        Put samples into matrix structure, just for representation
        :return: list of matrix-shaped samples identifiers
        """
        design = self.design_matrix()
        if np.where(self == '', False, True).all():
            pools_: List[str] = []
            for i in range(design.shape[0]):
                cache = (design[i, :].reshape(self.shape) == False)  # cache = ~design[i, :].reshape(self.shape)
                pool = np.ma.masked_array(self, mask=cache)
                pools_.append(pool.compressed())
            return pools_
            # return just for being able to print the list if wished

    def set_line_values(self, samples: list, variant: Variant,
                        sig: object = None,
                        params: List[float] = [], interp: object = None) -> None:
        """
        Attach sigmoid-transformed alternate allele frequencies to the current variant.
        :param samples: samples identifiers from the VCF-file
        :param variant: variant identifier from the VCF-file
        :param sig: sigmoid object computed from another set of pooled genotype data
        :param params: parameters to pass for the sigmoid approximation
        :param interp: interpolated sigmoid object for sigmoid approximation
        :return: variant object with attributes attached, e.g. twisted alternate allele frequency
        """
        self.__setattr__('variant', variant)  #variant.genotypes
        self.__setattr__('samples', samples)
#         self.__setattr__('var_pos', str(variant.POS))
#         self.__setattr__('aaf', variant.aaf)
#         if sig is not None:
#             self.__setattr__('aat', sig.call_sigmoid(params, self.aaf))
#             self.__setattr__('aat_', sig.call_sigmoid_derivative(interp, self.aaf))
#         else:
#             self.__setattr__('aat', np.nan)
#             self.__setattr__('aat_', np.nan)

    def get_call(self) -> np.ndarray:
        """
        Get pooled samples genotypes for a given variant
        :return:
        """
        subs = self.get_subset()
        idx = np.argwhere(np.isin(self.samples, subs))
        self.__setattr__('call', np.asarray(self.variant)[idx])
        return self.call

    def pool_genotypes(self) -> np.ndarray:
        """
        Computes genotypes of the different pools.
        :return: array of {0,1} GLs for RR|RA|AA for each pool
        """
        call: np.ndarray = self.get_call().reshape((1, self.size, 3)) #self.size=16
        scores: np.ndarray = np.apply_along_axis(sum, axis=-1, arr=call[:, :, :-1]) #sum of allele1+allel2

        if np.isin(call, -1).any():
            x = np.ones((1, self.pools_nb, 1))
            y = np.asarray([-1, -1, 0])
            b = np.broadcast(x, y)
            p = np.empty(b.shape)
            p.flat = [u * v for (u, v) in b]
        else:
            pooled_gt = np.dot(self.design_matrix(),
                               np.transpose(scores)).reshape((1, self.pools_nb, 1))
            pooled_gt = np.broadcast_to(pooled_gt, (1, self.pools_nb, 3))
            p = np.apply_along_axis(self.pooler_gt, axis=-1, arr=pooled_gt)

        return p  # list of gt for the 8 pools from design matrix

    def pooler_gt(self, a: np.ndarray) -> np.ndarray:
        """
        Decodes pooled scores into individual GT.
        :param a: score from matrix-vector pooling
        :return: pool's true genotype with phase
        """
        if np.all(a == 0):  # RR * RR * RR * RR
            gt = [0, 0, 0]
        elif np.all(a == self.pools_nb):  # AA * AA * AA * AA
            gt = [1, 1, 0]
        else:
            gt = [1, 0, 0]
        return gt

    def decode_genotypes_gt(self, samples_gt: np.ndarray) -> np.ndarray:
        """
        Recomputes true genotypes of samples with/without pooling/missing data
        :param pooled_samples: Variant.genotypes (unpooled samples' true genotypes with phase)
        :return: individual samples genotypes (true genotype with phase)
        """
        pooled: np.ndarray = self.pool_genotypes()  # pooled[:, :, -1]: bool = phase of the genotype
        scores: np.ndarray = np.apply_along_axis(sum, axis=-1, arr=pooled[:, :, :-1])
        p = np.argwhere(np.isin(self.samples, self.subset))

        count_alt: Callable[int, int] = lambda x: 1 if 1 in x else 0
        count_ref: Callable[int, int] = lambda x: 1 if 0 in x else 0

        alt_row: int = np.sum(np.apply_along_axis(count_alt,
                                                  axis=-1,
                                                  arr=pooled[:, :4, :-1]))
        alt_col: int = np.sum(np.apply_along_axis(count_alt,
                                                  axis=-1,
                                                  arr=pooled[:, 4:, :-1]))
        ref_row: int = np.sum(np.apply_along_axis(count_ref,
                                                  axis=-1,
                                                  arr=pooled[:, :4, :-1]))
        ref_col: int = np.sum(np.apply_along_axis(count_ref,
                                                  axis=-1,
                                                  arr=pooled[:, 4:, :-1]))

        nb_alt: int = alt_row + alt_col
        nb_ref: int = ref_row + ref_col

        if np.isin(pooled, -1).any():
            x = np.ones((1, self.size, 1))
            y = np.asarray([-1, -1, 0])
            b = np.broadcast(x, y)
            decoded_gt = np.empty(b.shape)
            decoded_gt.flat = [u * v for (u, v) in b]

        else:
            encoded = np.dot(scores,
                             self.design_matrix()).reshape(1, self.size, 1)
            b = np.broadcast_to(encoded, (1, self.size, 3))
            if nb_alt == 0:
                decoded_gt = np.zeros_like(b)
            elif nb_ref == 0:
                aa = np.array([1, 1, 0])
                decoded_gt = np.tile(aa, self.size).reshape((1, self.size, 3))
            elif nb_alt == 2:
                decoder: Callable = lambda x: [1, -1, 0] if np.all(x == 2) else [0, 0, 0]
                # np.all() because of b.shape
                decoded_gt = np.apply_along_axis(decoder, axis=-1, arr=b)
            elif nb_ref == 2:  # symmetric case with ref allele in only 2 pools: individual is RR or RA
                decoder: Callable = lambda x: [0, -1, 0] if np.all(x == 2) else [1, 1, 0]
                decoded_gt = np.apply_along_axis(decoder, axis=-1, arr=b)
            else:  # nb_alt > 2 and nb_alt < 8: # nb_alt = 2*n with n > 1
                decoded_gt = np.apply_along_axis(self.multidecoder_gt, axis=-1, arr=b)

        np.put_along_axis(samples_gt,
                          np.broadcast_to(p, (self.size, 3)),
                          decoded_gt.squeeze(),
                          axis=0)

        return samples_gt

    def decode_genotypes_gl(self, samples_gl: np.ndarray, dict_gl: dict) -> np.ndarray:
        """
        Recomputes genotypes likelihoods of samples with/without pooling/missing data
        :param samples_gl: samples' true genotypes with phase
        :param dict_gl: likelihoods values to set when encountering missing genotypes
        :return: individual samples genotypes (genotype likelihoods)
        """
        samples_gl = samples_gl.astype(float)  # avoid truncating GL
        pooled: np.ndarray = self.pool_genotypes()  # outputs unphased genotypes
        scores: np.ndarray = np.apply_along_axis(sum, axis=-1, arr=pooled[:, :, :-1]).flatten()
        p = np.argwhere(np.isin(self.samples, self.subset))
        count_alt: Callable[int] = lambda x: 1 if 1 in x else 0
        count_ref: Callable[int] = lambda x: 1 if 0 in x else 0

        alt_row: int = np.sum(np.apply_along_axis(count_alt,
                                                  axis=-1,
                                                  arr=pooled[:, :4, :-1]))  # :-1 excludes the phase boolean
        alt_col: int = np.sum(np.apply_along_axis(count_alt,
                                                  axis=-1,
                                                  arr=pooled[:, 4:, :-1]))
        ref_row: int = np.sum(np.apply_along_axis(count_ref,
                                                  axis=-1,
                                                  arr=pooled[:, :4, :-1]))
        ref_col: int = np.sum(np.apply_along_axis(count_ref,
                                                  axis=-1,
                                                  arr=pooled[:, 4:, :-1]))

        rowcounts, colcounts = self.rowcolcounts(scores)

        nb_alt: int = alt_row + alt_col
        nb_ref: int = ref_row + ref_col

        if np.isin(pooled, -1).any():
            x = np.ones((1, self.size, 1))
            y = np.asarray([1/3, 1/3, 1/3])
            b = np.broadcast(x, y)
            decoded_gl = np.empty(b.shape)
            decoded_gl.flat = [u * v for (u, v) in b]

        else:
            encoded = np.dot(scores,
                             self.design_matrix()).reshape(1, self.size, 1)
            b = np.broadcast_to(encoded, (1, self.size, 3)).astype(float)
            if nb_alt == 0:
                rr = np.array([1, 0, 0])
                decoded_gl = np.tile(rr, self.size).reshape((1, self.size, 3))
            elif nb_ref == 0:
                aa = np.array([0, 0, 1])
                decoded_gl = np.tile(aa, self.size).reshape((1, self.size, 3))
            else:  # nb_alt >= 2 and nb_alt < 8: # nb_alt = 2*n with n > 1
                unknown = dict_gl[tuple([*rowcounts, *colcounts, 1, 1])]  # 1,1 only cross-section that outputs unknown genotypes
                decoded_gl = np.apply_along_axis(self.multidecoder_gl, -1, b, unknown)

        np.put_along_axis(samples_gl,
                          np.broadcast_to(p, (self.size, 3)),
                          decoded_gl.squeeze(),
                          axis=0)

        return samples_gl

    @staticmethod
    def rowcolcounts(a: np.ndarray) -> Tuple[Tuple[int, int, int]]:
        """
        Count number of pooled RR|RA|AA genotypes over all rows and columns of a pooled matrix
        :param a: score i.e. trinary-encoded true genotypes
        :return: counts of genotypes for the rows and columns
        """
        # a should be scores
        count_rr: Callable[int] = np.vectorize(lambda x: 1 if x == 0 else 0)
        count_aa: Callable[int] = np.vectorize(lambda x: 1 if x == 2 else 0)
        count_ra: Callable[int] = np.vectorize(lambda x: 1 if x == 1 else 0)

        rowcounts: List[int] = [0, 0, 0]  # counts number of RR, RA, AA rows
        colcounts: List[int] = [0, 0, 0]  # counts number of RR, RA, AA columns
        rowcounts[0] = np.sum(count_rr(a[:4]))
        colcounts[0] = np.sum(count_rr(a[4:]))
        rowcounts[1] = np.sum(count_ra(a[:4]))
        colcounts[1] = np.sum(count_ra(a[4:]))
        rowcounts[2] = np.sum(count_aa(a[:4]))
        colcounts[2] = np.sum(count_aa(a[4:]))

        return tuple(rowcounts), tuple(colcounts)

    @staticmethod
    def multidecoder_gt(a: np.ndarray): ##deleted  -> prm.GLtype
        """
        Decodes pooled scores into individual GT.
        :param a: score
        :return: true genotype with phase
        """
        if np.all(a == 2):  # RA * RA
            gt = [-1, -1, 0]
        elif np.all(a == 1) or np.all(a == 0):  # RA * RR or RR * RR
            gt = [0, 0, 0]
        else:
            gt = [1, 1, 0]
        return gt

    @staticmethod
    def multidecoder_gl(a: np.ndarray, mis: List[float]): ##deleted  -> prm.GLtype
        """
        Decodes pooled scores into individual GL.
        :param a: score
        :param mis: GL to set when unknown genotype
        :return: genotypes likeihoods
        """
        if np.all(a == 2):  # RA * RA
            gl = mis
        elif np.all(a == 1) or np.all(a == 0):  # RA * RR or RR * RR
            gl = [1.0, 0.0, 0.0]
        else:
            gl = [0.0, 0.0, 1.0]
        return gl

    @staticmethod
    def normalize(v):
        """
        Normalize (Z-norm) an array. See NumPy documentation.
        :param v: array to normalize
        :return: normalized array
        """
        norm = np.linalg.norm(v)
        if norm == 0:
            return v
        return v / norm


    def __array_finalize__(self, obj: object) -> None:
        """
        Constructor needed for subclassing NumPy arrays.
        See online documentation.
        :param obj:
        :return:
        """
        if obj is None: return
        self.info = getattr(obj, 'info', None)

In [5]:
sample_id = list(['id' + str(i) for i in range(16)])   #generate id label of samples
sample_id

['id0',
 'id1',
 'id2',
 'id3',
 'id4',
 'id5',
 'id6',
 'id7',
 'id8',
 'id9',
 'id10',
 'id11',
 'id12',
 'id13',
 'id14',
 'id15']

In [6]:
# sample_gt = np.array([[[1, 1, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]],
#                [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]],
#                [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]],
#                [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]]])
sample_gt = np.array([[1, 1, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1],
               [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1],
               [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1],
               [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]])
sample_gt

array([[1, 1, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [7]:
mypooling = SNPsPool()
mypooling
# build am empty matrix for sample id

SNPsPool([['', '', '', ''],
          ['', '', '', ''],
          ['', '', '', ''],
          ['', '', '', '']], dtype='<U8')

In [8]:
mypooling.set_subset(sample_id)

SNPsPool([['id0', 'id1', 'id2', 'id3'],
          ['id4', 'id5', 'id6', 'id7'],
          ['id8', 'id9', 'id10', 'id11'],
          ['id12', 'id13', 'id14', 'id15']], dtype='<U8')

In [9]:
mypooling.pools_list() #each pools contains which subject

[SNPsPool(['id0', 'id1', 'id2', 'id3'], dtype='<U8'),
 SNPsPool(['id4', 'id5', 'id6', 'id7'], dtype='<U8'),
 SNPsPool(['id8', 'id9', 'id10', 'id11'], dtype='<U8'),
 SNPsPool(['id12', 'id13', 'id14', 'id15'], dtype='<U8'),
 SNPsPool(['id0', 'id4', 'id8', 'id12'], dtype='<U8'),
 SNPsPool(['id1', 'id5', 'id9', 'id13'], dtype='<U8'),
 SNPsPool(['id2', 'id6', 'id10', 'id14'], dtype='<U8'),
 SNPsPool(['id3', 'id7', 'id11', 'id15'], dtype='<U8')]

In [10]:
mypooling.get_subset() #flatten self._new_

SNPsPool(['id0', 'id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8',
          'id9', 'id10', 'id11', 'id12', 'id13', 'id14', 'id15'],
         dtype='<U8')

In [11]:
mypooling.design_matrix()    

# Matrix design
# There are 8 pools, 
# pool 1 is the first row, it contains item 1 2 3 4
# pool 5 is the fifth row, it contains item 1 5 9 13

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]])

In [12]:
mypooling.set_line_values(sample_id,sample_gt) #set value: id and gt

In [13]:
mypooling.get_call() #the original individual gt

array([[[1, 1, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]],

       [[0, 0, 1]]])

In [14]:
mypooling.pool_genotypes() #the pool's gt

array([[[1, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [1, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]])

In [15]:
mypooling.pooler_gt(np.ndarray([8,8,8])) 
# used by mypooling.pool_genotypes()
# rules to summerize individule info to pool info

[1, 0, 0]

In [16]:
mypooling.decode_genotypes_gt(sample_gt) #the retrieved individual gt

array([[ 1, -1,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0]])

In [17]:
mypooling.multidecoder_gt(np.ndarray([8,8,8])) #rule to retrive from pool info ro individual info

[1, 1, 0]